In [ ]:
import helpers
import pandas as pd
import time
import copy
from helpers import geodatarefined,langchainhelpers
from tqdm import tqdm
from langchain.prompts import ChatPromptTemplate
pd.set_option("display.max_colwidth",1000)

In [ ]:
output_parser=langchainhelpers.customOutputFunctionsParser()

In [ ]:
#load functions 
all_available_functions = {"searchdetails":geodatarefined.searchdetails,
             "get_city_details":geodatarefined.get_city_details,
             "get_country_details":geodatarefined.get_country_details,
             "get_state_details":geodatarefined.get_state_details,
             "get_airport_details":geodatarefined.get_airport_details,
             "get_university_details":geodatarefined.get_university_details,
             "get_attraction_details":geodatarefined.get_attraction_details,
             "search_open_street_maps":geodatarefined.search_open_street_maps,
             "calculate_distance":geodatarefined.calculate_distance}


In [ ]:
prompt2 = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a geography enthusiast and know a lot about this field."),
            ("human", "You also have access to a Geographical database, so make calls to the relevant functions if and only if necessary. Remember it costs to call the database and the service isn't cheap."),
            ("human", "{input}")
        ]
    )

In [ ]:
results=helpers.download_file("data/Geodata/intermediate_results.pkl")

# Experimentation

# 8 subfunctions available Prompt 2 

In [ ]:
domain_specific_questions_df=pd.read_csv("data/Geodata/Domain_specific_questions.csv")
domain_specific_questions_df

In [ ]:
available_functions={k:v for k,v in all_available_functions.items() if  v.__name__!="searchdetails"}
available_functions

In [ ]:
results=[]

In [ ]:
for question in tqdm(domain_specific_questions_df["Question"]): 
    prompt=copy.deepcopy(prompt2)
    results.append(langchainhelpers.run_llm_chain_once(prompt,available_functions,question))
    helpers.upload_file("data/GeoData/intermediate_results.pkl",results)

In [ ]:
results

In [ ]:
helpers.upload_file("data/Geodata/domain_specific_questions_with_8_subfunctions_available_prompt_2_with_better_descriptions.pkl",results)

In [ ]:
results=helpers.download_file("data/Geodata/domain_specific_questions_with_8_subfunctions_available_prompt_2_with_better_descriptions.pkl")
results=pd.DataFrame(results)
results=pd.merge(left=results,right=domain_specific_questions_df)
results

# 2 subfunctions available Prompt 2 

In [ ]:
domain_specific_questions_df=pd.read_csv("data/Geodata/Domain_specific_questions.csv")
domain_specific_questions_df

In [ ]:
available_functions={k:v for k,v in all_available_functions.items() if  v.__name__=="search_Open_Street_Maps" or v.__name__=="calculate_distance"}
available_functions

In [ ]:
results=[]

In [ ]:
for question in tqdm(domain_specific_questions_df["Question"]): 
    prompt=copy.deepcopy(prompt2)
    results.append(langchainhelpers.run_llm_chain_once(prompt,available_functions,question))
    helpers.upload_file("data/GeoData/intermediate_results.pkl",results)

In [ ]:
helpers.upload_file("data/Geodata/domain_specific_questions_with_2_subfunctions_available_prompt_2_with_better_descriptions.pkl",results)

In [ ]:
results=helpers.download_file("data/Geodata/domain_specific_questions_with_2_subfunctions_available_prompt_2_with_better_descriptions.pkl")
results=pd.DataFrame(results)
results=pd.merge(left=results,right=domain_specific_questions_df)
results

# 8 subfunctions available Prompt 2 (Multiple Calls)

In [ ]:
domain_specific_questions_df=pd.read_csv("data/Geodata/Domain_specific_questions.csv")
domain_specific_questions_df

In [ ]:
available_functions={k:v for k,v in all_available_functions.items() if  v.__name__!="searchdetails"}
available_functions

In [ ]:
results=[]

In [ ]:
for question in tqdm(domain_specific_questions_df["Question"]): 
    prompt=copy.deepcopy(prompt2)
    results.append(langchainhelpers.run_llm_chain_more_than_once(prompt,available_functions,question,1000,1,10))
    helpers.upload_file("data/GeoData/intermediate_results.pkl",results)

In [ ]:
results

In [ ]:
helpers.upload_file("data/Geodata/domain_specific_questions_with_8_subfunctions_available_prompt_2_with_better_descriptions_multiple_calls.pkl",results)

In [ ]:
results=helpers.download_file("data/Geodata/domain_specific_questions_with_8_subfunctions_available_prompt_2_with_better_descriptions_multiple_calls.pkl")
results=pd.DataFrame(results)
results=pd.merge(left=results,right=domain_specific_questions_df)
results

# 2 subfunctions available Prompt 2 (Multiple Calls)

In [ ]:
domain_specific_questions_df=pd.read_csv("data/Geodata/Domain_specific_questions.csv")
domain_specific_questions_df

In [ ]:
available_functions={k:v for k,v in all_available_functions.items() if  v.__name__=="search_Open_Street_Maps" or v.__name__=="calculate_distance"}
available_functions

In [ ]:
results=[]

In [ ]:
for question in tqdm(domain_specific_questions_df["Question"]): 
    prompt=copy.deepcopy(prompt2)
    results.append(langchainhelpers.run_llm_chain_more_than_once(prompt,available_functions,question,1000,1,10))
    helpers.upload_file("data/GeoData/intermediate_results.pkl",results)

In [ ]:
results

In [ ]:
helpers.upload_file("data/Geodata/domain_specific_questions_with_2_subfunctions_available_prompt_2_with_better_descriptions_multiple_calls.pkl",results)

In [ ]:
results=helpers.download_file("data/Geodata/domain_specific_questions_with_2_subfunctions_available_prompt_2_with_better_descriptions_multiple_calls.pkl")
results=pd.DataFrame(results)
results=pd.merge(left=results,right=domain_specific_questions_df)
results

In [ ]:
def create_ground_truth_function_call(steps):
    ground_truth_function_call=""
    for step in steps:
        ground_truth_function_call= ground_truth_function_call + step["Function_name"]+ \
            str(step["Function_arguments"]).replace("{","(")\
            .replace("}",")").replace(":","=")+", "
    return ground_truth_function_call

results["Correct Function Call"]=results["steps"].apply(lambda x: create_ground_truth_function_call(x) )
results[["Question","Correct Answer","Correct Function Call"]]